In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
import random
import os
from dotenv import load_dotenv

# Cargamos las variables de entorno
load_dotenv(dotenv_path='./../variables.env')

# set the seed
seed = int(os.getenv('SEED'))
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
def linearize(model):
  """ 
    Get the absolute value of the weights of the model and return the signs of the weights 
  """
  signs = []
  for layer in model.layers:
    if len(layer.get_weights()) > 0:
      weights = layer.get_weights()
      layer_signs = [np.sign(w) for w in weights]
      new_weights = [np.abs(w) for w in weights]
      layer.set_weights(new_weights)
      signs.append(layer_signs)
  return signs

In [3]:
def compute_synflow_per_weight(model, input_shape):
  # Añadimos la dimensión del batch en input_shape
  batch_input_shape = (1,) + input_shape
  
  # Genera un input aleatorio con la forma correcta
  random_input = tf.random.normal(batch_input_shape)
  
  with tf.GradientTape() as tape:
      # Realiza una pasada hacia adelante para obtener la salida de la red
      output = model(random_input)
      
      # Calcula la suma de los outputs para obtener un escalar
      loss = tf.reduce_sum(output)
  
  # Calcula los gradientes de la "loss" con respecto a los pesos del modelo
  gradients = tape.gradient(loss, model.trainable_variables)

  # Muestra los gradientes
  print("Gradients:")
  print(gradients)
  
  # Calcula los scores (producto elemento a elemento de los pesos y los gradientes)
  scores = [tf.abs(weight * grad) for weight, grad in zip(model.trainable_variables, gradients)]
  
  return scores

In [6]:
# Example model with ones inizialitation
conv_args = {
  "activation": "relu",
  "padding": "same",
  "kernel_initializer": "ones",
  "bias_initializer": "ones"
}

model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(32, kernel_size=(1, 1), input_shape=(64, 64, 1),  **conv_args),
  tf.keras.layers.Conv2D(64, (3, 3), **conv_args),
  tf.keras.layers.Conv2D(64, (3, 3), **conv_args),
  tf.keras.layers.Conv2D(64, (3, 3), **conv_args),
  tf.keras.layers.Conv2D(64, (3, 3), **conv_args),
  tf.keras.layers.Conv2D(64, (3, 3), **conv_args)
])

# initialize model weights
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.build()

# Define the input shape
input_shape = (64, 64, 1)

# Compute synflow scores
synflow_scores = compute_synflow_per_weight(model, input_shape)

# Print the synflow scores
for i, score in enumerate(synflow_scores):
  print(f"Synflow score of layer {i}: {score}")

# Print the synflow scores summed over all weights for each layer
total_synflow_scores = [np.sum(score) for score in synflow_scores]
for i, score_sum in enumerate(total_synflow_scores):
  print(f"Total synflow score of layer {i}: {score_sum}")

# Print the total synflow score
total_synflow_score = np.sum(total_synflow_scores)
print(f"Total synflow score: {total_synflow_score}")

Gradients:
[<tf.Tensor: shape=(1, 1, 1, 32), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.]]]], dtype=float32)>, <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>, <tf.Tensor: shape=(3, 3, 32, 64), dtype=float32, numpy=
array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0

In [5]:
from tensorflow import keras
import prettytable as pt
from dotenv import load_dotenv

# Cargamos las variables de entorno
load_dotenv(dotenv_path='./../variables.env')

seed = int(os.getenv("SEED"))

def set_seed():
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

#set_seed()


# Modelo con inicialización y activación revisadas
model_bad = keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(33, activation='relu', kernel_initializer='he_normal', bias_initializer='zeros'),
    layers.Dense(88, activation='relu', kernel_initializer='he_normal', bias_initializer='zeros'),
    layers.Dense(10, activation='softmax')
])

# Compilamos el modelo
model_bad.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Construimos el modelo
model_bad.build()

# Calcula los SynFlow scores usando el código previamente ajustado
score_model_bad = compute_synflow_per_weight(model_bad, (28, 28))

table = pt.PrettyTable()
table.field_names = ["Layer", "Bad model"]
for i, score in enumerate(score_model_bad):
  table.add_row([f"Layer {i}", np.sum(score)])
print(table)

score_model_bad_sum = np.sum([np.sum(score) for score in score_model_bad])

print(f"Total synflow score of bad model: {score_model_bad_sum}")

Gradients:
[<tf.Tensor: shape=(784, 33), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>, <tf.Tensor: shape=(33,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>, <tf.Tensor: shape=(33, 88), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>, <tf.Tensor: shape=(88,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0

C:\Users\Jafet\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
